Data engineering

This notebook describes the process to download and prepare United States presidential election data. This notebook addresses missing values, reformats data types, and restructures the format of the data tables.

To download and prepare the election data, ArcPy, ArcGIS API for Python, and pandas modules are used.

Install geopandas

In [4]:
#@title
# Install OpticalRS. This also installs GeoPandas because GeoPandas is a dependency of OpticalRS
!apt-get install software-properties-common python-software-properties > /dev/null
!add-apt-repository ppa:ubuntugis/ppa -y > /dev/null
!apt-get update > /dev/null
!apt-get install -y --fix-missing python-gdal gdal-bin libgdal-dev > /dev/null
!pip2 install OpticalRS > /dev/null

! pip install geopandas

E: Package 'python-software-properties' has no installation candidate
E: Package 'python-gdal' has no installation candidate
/bin/bash: line 1: pip2: command not found


##### Import needed modules

In [5]:
import pandas as pd
import geopandas as gpd
import os



## Load and prepare election data

When reading the CSV into a pandas data frame, it is necessary to ensure that the county_fips field is read as an object, rather than a number, to prevent deletion of the leading 0 character used by some counties.

##### Read data into Python

In [91]:
# The CSV file is in the same folder as this notebook; providing the CSV file name is sufficient.
selections_data_path = "https://raw.githubusercontent.com/SergeyShchus/Data-analysis-Notebooks/main/Data/countypres_2000-2020.csv"
selections_complete_df = pd.read_csv(selections_data_path)

Some of the table headings could be improved to better identify the data in the column. You will rename these fields and; then use the pandas head function to return the first five rows of the table to confirm that the new headings are correct.

In [92]:
rename_cols = {
    "state_po": "state_abbr",
    "county_fips": "FIPS",
    "party": "pol_identity"
}
selections_complete_df.rename(columns=rename_cols, inplace=True)
selections_complete_df.head()

,year,state,state_abbr,county_name,FIPS,office,candidate,pol_identity,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL


The election data includes records that are missing data in the FIPS field. This missing data is referred to as null values. You will later join this table to voting-age population data based on the FIPS value, so this value cannot be null. You will identify how many rows have null values and establish a strategy for dealing with those null values.

In [93]:
# Find records with null values.
selections_complete_df.query("FIPS.isnull()")

,year,state,state_abbr,county_name,FIPS,office,candidate,pol_identity,candidatevotes,totalvotes,version,mode
12452,2000,CONNECTICUT,CT,STATEWIDE WRITEIN,NaN,US PRESIDENT,AL GORE,DEMOCRAT,0,0,20220315,TOTAL
12453,2000,MAINE,ME,MAINE UOCAVA,NaN,US PRESIDENT,AL GORE,DEMOCRAT,0,0,20220315,TOTAL
12454,2000,RHODE ISLAND,RI,FEDERAL PRECINCT,NaN,US PRESIDENT,AL GORE,DEMOCRAT,0,0,20220315,TOTAL
12455,2000,CONNECTICUT,CT,STATEWIDE WRITEIN,NaN,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,0,0,20220315,TOTAL
12456,2000,MAINE,ME,MAINE UOCAVA,NaN,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,0,0,20220315,TOTAL
12457,2000,RHODE ISLAND,RI,FEDERAL PRECINCT,NaN,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,0,0,20220315,TOTAL
12458,2000,CONNECTICUT,CT,STATEWIDE WRITEIN,NaN,US PRESIDENT,RALPH NADER,GREEN,0,0,20220315,TOTAL
12459,2000,MAINE,ME,MAINE UOCAVA,NaN,US PRESIDENT,RALPH NADER,GREEN,0,0,20220315,TOTAL
12460,2000,RHODE ISLAND,RI,FEDERAL PRECINCT,NaN,US PRESIDENT,RALPH NADER,GREEN,0,0,20220315,TOTAL
12461,2000,CONNECTICUT,CT,STATEWIDE WRITEIN,NaN,US PRESIDENT,OTHER,OTHER,0,0,20220315,TOTAL


After reviewing the records, you notice that some of the records represent votes cast in Washington, DC, for the 2020 election. Removing these records would affect hundreds of thousands of votes and make it impossible to include Washington, DC, in the analysis for 2020. Additional research indicates that the appropriate FIPS code for Washington, DC, is 11001. You will add the correct FIPS code for Washington, DC, records.

In [94]:
# Add the FIPS code for Washington, DC.
selections_complete_df.loc[selections_complete_df['state_abbr'] == 'DC', 'FIPS'] = '11001'

The remaining records with null FIPS values are votes assigned at a state-wide level, not to any county. Your analysis will take place at the county level, which means that these records cannot be used. You will assess the effect that removing these votes will have on your overall analysis.

In [95]:
# Determine the total number of records in the table.
rowcount = selections_complete_df.shape[0]

# Determine the total number of votes cast across all counties in all elections.
votecount = selections_complete_df["candidatevotes"].sum()

# Determine how many votes are associated with null FIPS values.
null_fips_df = selections_complete_df.query("FIPS.isnull()")
null_fips_rowcount = null_fips_df.shape[0]
null_fips_votecount = null_fips_df["candidatevotes"].sum()

# Calculate the percentage of the data affected by null records.
percentage_null_fips_rows = round((null_fips_rowcount / rowcount) * 100, 3)
percentage_null_fips_votes = round((null_fips_votecount / votecount) * 100, 3)

# Use a print statement to report this information.
print(f"There were {null_fips_rowcount} records with null FIPS values in the data, representing {null_fips_votecount} votes.")
print(f"This amounts to {percentage_null_fips_rows}% of the total records and {percentage_null_fips_votes}% of the total votes.")

There were 52 records with null FIPS values in the data, representing 13009 votes.
This amounts to 0.072% of the total records and 0.002% of the total votes.


The records with null FIPS values represent a small percentage of the total, and they are unlikely to change the overall analysis. You will create a new dataframe that includes only records with non-null FIPS values.

In [96]:
# Remove records with null FIPS values.
elections_df = selections_complete_df.query("FIPS.notnull()")

Even after correcting for the missing FIPS values, the format of the election data table will prevent a proper join to the voting-age population data. Currently, each record in the table corresponds to a candidate's votes for each county for each vote mode. You need to reformat the table so that each record corresponds to a single county for each election year, with fields showing the total votes for each major party candidate for that election year in that county. It is possible to do this using the [Pivot Table geoprocessing tool](https://pro.arcgis.com/en/pro-app/tool-reference/data-management/pivot-table.htm) or Excel pivot tables, but Python may make it easier to automate and share.

The steps in restructuring the table are as follows:

1. Limit records to only candidates who identify as either a Democrat or a Republican
2. Sum each candidate's votes for every mode in each county, for each election year
3. Lock some fields in place, and then pivot the table using the party field.
4. Reformat the pivoted table.

The following code cell performs these steps.
![reformat_table.gif](attachment:reformat_table.gif)

In [97]:
# Filter records by political identity for Democrats and Republicans.
selections_df = elections_df.query("pol_identity in ['DEMOCRAT', 'REPUBLICAN']")

In [98]:
# Group records for each candidate by county and year.
candidate_group = elections_df.groupby(['FIPS', 'county_name', 'state', 'candidate', 'year', 'pol_identity', 'totalvotes'])

# Sum each candidate's votes in each county, for each election year.
candidate_votes = candidate_group['candidatevotes'].sum()

# Remove the multi-index created by the group-by and sum, returning a new data frame.
candidate_votes_df = candidate_votes.reset_index()

# Check the output.
candidate_votes_df.head()

,FIPS,county_name,state,candidate,year,pol_identity,totalvotes,candidatevotes
0,1001.0,AUTAUGA,ALABAMA,AL GORE,2000,DEMOCRAT,17208,4942
1,1001.0,AUTAUGA,ALABAMA,BARACK OBAMA,2008,DEMOCRAT,23641,6093
2,1001.0,AUTAUGA,ALABAMA,BARACK OBAMA,2012,DEMOCRAT,23932,6363
3,1001.0,AUTAUGA,ALABAMA,DONALD J TRUMP,2020,REPUBLICAN,27770,19838
4,1001.0,AUTAUGA,ALABAMA,DONALD TRUMP,2016,REPUBLICAN,24973,18172


In [99]:
# Pivot the data frame.
# The index "locks" the fields that remain the same for each county.
# The columns determine which values will become new fields; in this case, the two values in the pol_identity column become two new fields.
# The values column determines which values will be reported for each of the new fields.
selections_pivot_df = candidate_votes_df.pivot(
    index=['year', 'FIPS', 'county_name', 'state', 'totalvotes'],
    columns=['pol_identity'],
    values=['candidatevotes']
)

# Check the output.
selections_pivot_df.head()

candidatevotes                      \
pol_identity                                     DEMOCRAT   GREEN LIBERTARIAN   
year FIPS   county_name state   totalvotes                                      
2000 1001.0 AUTAUGA     ALABAMA 17208              4942.0   160.0         NaN   
     1003.0 BALDWIN     ALABAMA 56480             13997.0  1033.0         NaN   
     1005.0 BARBOUR     ALABAMA 10395              5188.0    46.0         NaN   
     1007.0 BIBB        ALABAMA 7101               2710.0    52.0         NaN   
     1009.0 BLOUNT      ALABAMA 17973              4977.0   154.0         NaN   

                                                              
pol_identity                                OTHER REPUBLICAN  
year FIPS   county_name state   totalvotes                    
2000 1001.0 AUTAUGA     ALABAMA 17208       113.0    11993.0  
     1003.0 BALDWIN     ALABAMA 56480       578.0    40872.0  
     1005.0 BARBOUR     ALABAMA 10395        65.0     5096.0  
     1007.0 BIBB        ALABAMA 7101         66.0     4273.0  
     1009.0 BLOUNT      ALABAMA 17973       175.0    12667.0

In [100]:
# Remove the multi-index since you no longer need these fields to be "locked" for the pivot.
selections_pivot_df.columns = selections_pivot_df.columns.get_level_values(1).rename(None)
selections_pivot_df.reset_index(inplace=True)

# Rename columns to better reflect their new meaning.
selections_pivot_df.rename(columns={"DEMOCRAT": "votes_dem","REPUBLICAN": "votes_gop"}, inplace=True)

# Check the output.
selections_pivot_df.head()

,year,FIPS,county_name,state,totalvotes,votes_dem,GREEN,LIBERTARIAN,OTHER,votes_gop
0,2000,1001.0,AUTAUGA,ALABAMA,17208,4942.0,160.0,NaN,113.0,11993.0
1,2000,1003.0,BALDWIN,ALABAMA,56480,13997.0,1033.0,NaN,578.0,40872.0
2,2000,1005.0,BARBOUR,ALABAMA,10395,5188.0,46.0,NaN,65.0,5096.0
3,2000,1007.0,BIBB,ALABAMA,7101,2710.0,52.0,NaN,66.0,4273.0
4,2000,1009.0,BLOUNT,ALABAMA,17973,4977.0,154.0,NaN,175.0,12667.0


Pandas has three powerful capabilities that helped perform this operation:

- The ability to perform an operation using a "groupby" function
- The ability to set an index using multiple fields, which acts as the "locking" mechanism
- The ability to unstack (or pivot) a table

## Load and prepare voting-age population data

The data includes multiple CSV files, each with estimates of the number of citizens voting in each county for a particular time period. You will load each CSV file into a data frame.

A dictionary is a good data structure to use to keep track of multiple related datasets.

In [101]:
# Create a dictionary of CSV files for each election year.
cvap_paths = {
    2020: "https://raw.githubusercontent.com/SergeyShchus/Data-analysis-Notebooks/main/Data/CountyCVAP_2017-2021.csv",
    2016: "https://raw.githubusercontent.com/SergeyShchus/Data-analysis-Notebooks/main/Data/CountyCVAP_2014-2018.csv",
    2012: "https://raw.githubusercontent.com/SergeyShchus/Data-analysis-Notebooks/main/Data/CountyCVAP_2010-2014.csv",
    2008: "https://raw.githubusercontent.com/SergeyShchus/Data-analysis-Notebooks/main/Data/CountyCVAP_2006-2010.csv"
}

# Create a dictionary of data frames for each election year.
cvap_dfs = {year:pd.read_csv(path, encoding="latin-1") for year, path in cvap_paths.items()}

# Display the first three rows of each data frame to check the output.
for df in cvap_dfs.values():
    display(df.head(3))

,geoname,lntitle,geoid,lnnumber,tot_est,tot_moe,adu_est,adu_moe,cit_est,cit_moe,cvap_est,cvap_moe
0,"Autauga County, Alabama",Total,0500000US01001,1,58240,0,44440,122,57625,310,43905,278
1,"Autauga County, Alabama",Not Hispanic or Latino,0500000US01001,2,56465,0,43255,123,56100,258,42965,223
2,"Autauga County, Alabama",American Indian or Alaska Native Alone,0500000US01001,3,100,86,100,86,100,86,100,86


,geoname,lntitle,geoid,lnnumber,tot_est,tot_moe,adu_est,adu_moe,cit_est,cit_moe,cvap_est,cvap_moe
0,"Autauga County, Alabama",Total,05000US01001,1,55200,0,41830,33,54665,173,41305,174
1,"Autauga County, Alabama",Not Hispanic or Latino,05000US01001,2,53670,0,40835,32,53395,123,40565,125
2,"Autauga County, Alabama",American Indian or Alaska Native Alone,05000US01001,3,160,68,85,46,160,68,85,46


,GEONAME,LNTITLE,GEOID,LNNUMBER,TOT_EST,TOT_MOE,ADU_EST,ADU_MOE,CIT_EST,CIT_MOE,CVAP_EST,CVAP_MOE
0,"Autauga County, Alabama",Total,05000US01001,1,55135,NaN,40920,NaN,54675,198,40545,160
1,"Autauga County, Alabama",Not Hispanic or Latino,05000US01001,2,53745,NaN,40055,NaN,53510,152,39885,115
2,"Autauga County, Alabama",American Indian or Alaska Native Alone,05000US01001,3,235,64.0,190,55.0,235,64,190,55


,GEONAME,LNTITLE,GEOID,LNNUMBER,TOT_EST,TOT_MOE,ADU_EST,ADU_MOE,CIT_EST,CIT_MOE,CVAP_EST,CVAP_MOE
0,"Autauga County, Alabama",Total,05000US01001,1,53155,NaN,38430,NaN,52540,250,38010,158
1,"Autauga County, Alabama",Not Hispanic or Latino,05000US01001,2,51920,NaN,37685,NaN,51610,148,37480,113
2,"Autauga County, Alabama",American Indian or Alaska Native Alone,05000US01001,3,160,82.0,135,72.0,160,82,135,72


The four separate CSV files resulted in four separate data frames, with some differences in the column headings. For this analysis, you will want a single table with estimates for total number of citizens of voting age for each year. You will also need a FIPS value to join this table to the election data above and the geoname to spotcheck data. To accomplish these goals, you will perform these steps:

1. Change all column headings to lowercase formatting so that each data frame has the exact same structure.
0. Add a year value to each of the data frames to differentiate their values.
0. Include only total count rows.
0. Remove any columns other than geoid, cvap_est, and year.
0. Extract the FIPS value from the geoid value.
0. Concatenate the data frames together.

In [102]:
# Create a new dictionary to hold the processed dataframes.
cvap_processed_dfs = {}

for year, df in cvap_dfs.items():

    # Set column formatting to lowercase.
    df.columns = df.columns.str.lower()

    # Add a year value.
    df['year'] = year

    # Include only rows of total counts.
    df = df.query("lntitle == 'Total'")

    # Include only necessary columns.
    df = df[['year', 'geoid','geoname','cvap_est']]

    # Extract FIPS from the geoid value.
    df['FIPS'] = df['geoid'].str[-5:]

    # Add processed data frames to the dictionary.
    cvap_processed_dfs[year] = df

# Concatenate the data frames.
cvap_df = pd.concat(cvap_processed_dfs.values())

# Check the output.
cvap_df

,year,geoid,geoname,cvap_est,FIPS
0,2020,0500000US01001,"Autauga County, Alabama",43905,01001
13,2020,0500000US01003,"Baldwin County, Alabama",174685,01003
26,2020,0500000US01005,"Barbour County, Alabama",19625,01005
39,2020,0500000US01007,"Bibb County, Alabama",17680,01007
52,2020,0500000US01009,"Blount County, Alabama",43635,01009
...,...,...,...,...,...
41808,2008,05000US72145,"Vega Baja Municipio, Puerto Rico",44270,72145
41821,2008,05000US72147,"Vieques Municipio, Puerto Rico",6970,72147
41834,2008,05000US72149,"Villalba Municipio, Puerto Rico",18830,72149
41847,2008,05000US72151,"Yabucoa Municipio, Puerto Rico",28295,72151


## Join tables

Now that both the voting-age population (cvap_df) and the election data (elections_pivot_df) tables have been prepared, they can be joined together. The combination of year and FIPS values for each county identifies a unique record, so these fields will be used to join the tables.

In [103]:
voting_info_df = pd.merge(selections_pivot_df, cvap_df, left_on=['FIPS', 'year'], right_on=['FIPS', 'year'], how='left')

# Check the output.
voting_info_df

,year,FIPS,county_name,state,totalvotes,votes_dem,GREEN,LIBERTARIAN,OTHER,votes_gop,geoid,geoname,cvap_est
0,2000,1001.0,AUTAUGA,ALABAMA,17208,4942.0,160.0,NaN,113.0,11993.0,NaN,NaN,NaN
1,2000,1003.0,BALDWIN,ALABAMA,56480,13997.0,1033.0,NaN,578.0,40872.0,NaN,NaN,NaN
2,2000,1005.0,BARBOUR,ALABAMA,10395,5188.0,46.0,NaN,65.0,5096.0,NaN,NaN,NaN
3,2000,1007.0,BIBB,ALABAMA,7101,2710.0,52.0,NaN,66.0,4273.0,NaN,NaN,NaN
4,2000,1009.0,BLOUNT,ALABAMA,17973,4977.0,154.0,NaN,175.0,12667.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18923,2020,56039.0,TETON,WYOMING,14787,9848.0,NaN,255.0,343.0,4341.0,NaN,NaN,NaN
18924,2020,56041.0,UINTA,WYOMING,9459,1591.0,NaN,172.0,200.0,7496.0,NaN,NaN,NaN
18925,2020,56043.0,WASHAKIE,WYOMING,4032,651.0,NaN,65.0,71.0,3245.0,NaN,NaN,NaN
18926,2020,56045.0,WESTON,WYOMING,3560,360.0,NaN,46.0,47.0,3107.0,NaN,NaN,NaN


You notice that the missing values from the voting-age population table are for the year 2000. The earliest voting-age population data that is available starts at the 2008 election. You will remove election data prior to 2008.

In [104]:
voting_info_df = voting_info_df.query("year >= 2008")

# Check the output.
voting_info_df

,year,FIPS,county_name,state,totalvotes,votes_dem,GREEN,LIBERTARIAN,OTHER,votes_gop,geoid,geoname,cvap_est
6309,2008,1001.0,AUTAUGA,ALABAMA,23641,6093.0,NaN,NaN,145.0,17403.0,NaN,NaN,NaN
6310,2008,1003.0,BALDWIN,ALABAMA,81413,19386.0,NaN,NaN,756.0,61271.0,NaN,NaN,NaN
6311,2008,1005.0,BARBOUR,ALABAMA,11630,5697.0,NaN,NaN,67.0,5866.0,NaN,NaN,NaN
6312,2008,1007.0,BIBB,ALABAMA,8644,2299.0,NaN,NaN,83.0,6262.0,NaN,NaN,NaN
6313,2008,1009.0,BLOUNT,ALABAMA,24267,3522.0,NaN,NaN,356.0,20389.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18923,2020,56039.0,TETON,WYOMING,14787,9848.0,NaN,255.0,343.0,4341.0,NaN,NaN,NaN
18924,2020,56041.0,UINTA,WYOMING,9459,1591.0,NaN,172.0,200.0,7496.0,NaN,NaN,NaN
18925,2020,56043.0,WASHAKIE,WYOMING,4032,651.0,NaN,65.0,71.0,3245.0,NaN,NaN,NaN
18926,2020,56045.0,WESTON,WYOMING,3560,360.0,NaN,46.0,47.0,3107.0,NaN,NaN,NaN


The records at the top and bottom of the data frame now look correct, but you will check whether there are any other null values for the cvap_est field.

In [105]:
voting_info_df.query("cvap_est.isnull()")

,year,FIPS,county_name,state,totalvotes,votes_dem,GREEN,LIBERTARIAN,OTHER,votes_gop,geoid,geoname,cvap_est
6309,2008,1001.0,AUTAUGA,ALABAMA,23641,6093.0,NaN,NaN,145.0,17403.0,NaN,NaN,NaN
6310,2008,1003.0,BALDWIN,ALABAMA,81413,19386.0,NaN,NaN,756.0,61271.0,NaN,NaN,NaN
6311,2008,1005.0,BARBOUR,ALABAMA,11630,5697.0,NaN,NaN,67.0,5866.0,NaN,NaN,NaN
6312,2008,1007.0,BIBB,ALABAMA,8644,2299.0,NaN,NaN,83.0,6262.0,NaN,NaN,NaN
6313,2008,1009.0,BLOUNT,ALABAMA,24267,3522.0,NaN,NaN,356.0,20389.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18922,2020,56037.0,SWEETWATER,WYOMING,16698,3823.0,NaN,350.0,296.0,12229.0,NaN,NaN,NaN
18923,2020,56039.0,TETON,WYOMING,14787,9848.0,NaN,255.0,343.0,4341.0,NaN,NaN,NaN
18924,2020,56041.0,UINTA,WYOMING,9459,1591.0,NaN,172.0,200.0,7496.0,NaN,NaN,NaN
18925,2020,56043.0,WASHAKIE,WYOMING,4032,651.0,NaN,65.0,71.0,3245.0,NaN,NaN,NaN


The election records for Alaska appear unusual because they are referencing a district rather than a county. Additional research shows that Alaska has a unique government model that does not use counties. The election data is reported for regions that do not correspond to the counties in the voting-age population table.

Because of this data incompatibility, you will remove records from the state of Alaska.

In [106]:
voting_info_df = voting_info_df.query("state != 'ALASKA'")

Next, you will query the data frame for any remaining records with null cvap_est values.

In [107]:
voting_info_df.query("cvap_est.isnull()")

,year,FIPS,county_name,state,totalvotes,votes_dem,GREEN,LIBERTARIAN,OTHER,votes_gop,geoid,geoname,cvap_est
6309,2008,1001.0,AUTAUGA,ALABAMA,23641,6093.0,NaN,NaN,145.0,17403.0,NaN,NaN,NaN
6310,2008,1003.0,BALDWIN,ALABAMA,81413,19386.0,NaN,NaN,756.0,61271.0,NaN,NaN,NaN
6311,2008,1005.0,BARBOUR,ALABAMA,11630,5697.0,NaN,NaN,67.0,5866.0,NaN,NaN,NaN
6312,2008,1007.0,BIBB,ALABAMA,8644,2299.0,NaN,NaN,83.0,6262.0,NaN,NaN,NaN
6313,2008,1009.0,BLOUNT,ALABAMA,24267,3522.0,NaN,NaN,356.0,20389.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18922,2020,56037.0,SWEETWATER,WYOMING,16698,3823.0,NaN,350.0,296.0,12229.0,NaN,NaN,NaN
18923,2020,56039.0,TETON,WYOMING,14787,9848.0,NaN,255.0,343.0,4341.0,NaN,NaN,NaN
18924,2020,56041.0,UINTA,WYOMING,9459,1591.0,NaN,172.0,200.0,7496.0,NaN,NaN,NaN
18925,2020,56043.0,WASHAKIE,WYOMING,4032,651.0,NaN,65.0,71.0,3245.0,NaN,NaN,NaN


You notice two more issues with the data. Additional research explains each of these issues:

* Kansas City, MO (FIPS 36000), reports its election data separately from Jackson County, MO (FIPS 29095).
* The independent city of Bedford, VA (FIPS 51515), merged with Bedford County, VA (FIPS 51019), in 2013. The voting-age population data used for the 2012 and 2016 elections was published after the merger and does not include separate data for the city of Bedford.

You can verify the research by querying the records of all four FIPS codes.

In [108]:
voting_info_df.query("FIPS in ['29095', '36000', '51019', '51515']")

,year,FIPS,county_name,state,totalvotes,votes_dem,GREEN,LIBERTARIAN,OTHER,votes_gop,geoid,geoname,cvap_est


To fix these issues, you will merge the data for each city into its respective county. The Pandas groupby function can pair each county with its respective city for each year and then sum the values of each pair together.

In [109]:
# Because the counties to fix are in different states, each group of a county and its associated city can be defined by year and state.
county_groups = voting_info_df.query("FIPS in ['29095', '36000', '51019', '51515']").groupby(['year', 'state'])
summed_votes = county_groups.sum()

# Match index values for summed data with original data.
summed_votes.index = voting_info_df.query("FIPS in ['29095', '51019']").sort_values('year').index

# Check the output.
summed_votes

<ipython-input-109-db9192262163>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  summed_votes = county_groups.sum()


,totalvotes,votes_dem,GREEN,LIBERTARIAN,OTHER,votes_gop,cvap_est


In [29]:
# Update county records with new summed values.
voting_info_df.loc[summed_votes.index, summed_votes.columns] = summed_votes

# Eliminate Kansas City and Bedford records.
voting_info_df = voting_info_df.query("FIPS not in ['36000', '51515']")

# Check the output.
voting_info_df.query("FIPS in ['29095', '36000', '51019', '51515']")

,year,FIPS,county_name,state,totalvotes,votes_dem,votes_gop,geoid,geoname,cvap_est


The county records now show the correct sum, and the records for Kansas City and the city of Bedford have been removed from the data frame.

In [110]:
# Verify that no record in the output data frame has a null cvap_est value.
voting_info_df.query("cvap_est.isnull()")

,year,FIPS,county_name,state,totalvotes,votes_dem,GREEN,LIBERTARIAN,OTHER,votes_gop,geoid,geoname,cvap_est
6309,2008,1001.0,AUTAUGA,ALABAMA,23641,6093.0,NaN,NaN,145.0,17403.0,NaN,NaN,NaN
6310,2008,1003.0,BALDWIN,ALABAMA,81413,19386.0,NaN,NaN,756.0,61271.0,NaN,NaN,NaN
6311,2008,1005.0,BARBOUR,ALABAMA,11630,5697.0,NaN,NaN,67.0,5866.0,NaN,NaN,NaN
6312,2008,1007.0,BIBB,ALABAMA,8644,2299.0,NaN,NaN,83.0,6262.0,NaN,NaN,NaN
6313,2008,1009.0,BLOUNT,ALABAMA,24267,3522.0,NaN,NaN,356.0,20389.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18922,2020,56037.0,SWEETWATER,WYOMING,16698,3823.0,NaN,350.0,296.0,12229.0,NaN,NaN,NaN
18923,2020,56039.0,TETON,WYOMING,14787,9848.0,NaN,255.0,343.0,4341.0,NaN,NaN,NaN
18924,2020,56041.0,UINTA,WYOMING,9459,1591.0,NaN,172.0,200.0,7496.0,NaN,NaN,NaN
18925,2020,56043.0,WASHAKIE,WYOMING,4032,651.0,NaN,65.0,71.0,3245.0,NaN,NaN,NaN


## Calculate additional columns

You will use the values from the updated table to add additional columns of information, such as the number of voters who did not choose the Democratic or Republican party, the percentage of voters for each party, and so on. Each column is referred to as an attribute of the dataset.

In [111]:
# Calculate voters who did not choose the Democratic or Republican party.
voting_info_df['votes_other'] = voting_info_df['totalvotes'] - (voting_info_df['votes_dem'] + voting_info_df['votes_gop'])

# Calculate voter share attributes.
voting_info_df['voter_share_major_party'] = (voting_info_df['votes_dem'] + voting_info_df['votes_gop']) / voting_info_df['totalvotes']
voting_info_df['voter_share_dem'] = voting_info_df['votes_dem'] / voting_info_df['totalvotes']
voting_info_df['voter_share_gop'] = voting_info_df['votes_gop'] / voting_info_df['totalvotes']
voting_info_df['voter_share_other'] = voting_info_df['votes_other'] / voting_info_df['totalvotes']

# Calculate raw difference attributes.
voting_info_df['rawdiff_dem_vs_gop'] = voting_info_df['votes_dem'] - voting_info_df['votes_gop']
voting_info_df['rawdiff_gop_vs_dem'] = voting_info_df['votes_gop'] - voting_info_df['votes_dem']
voting_info_df['rawdiff_dem_vs_other'] = voting_info_df['votes_dem'] - voting_info_df['votes_other']
voting_info_df['rawdiff_gop_vs_other'] = voting_info_df['votes_gop'] - voting_info_df['votes_other']
voting_info_df['rawdiff_other_vs_dem'] = voting_info_df['votes_other'] - voting_info_df['votes_dem']
voting_info_df['rawdiff_other_vs_gop'] = voting_info_df['votes_other'] - voting_info_df['votes_gop']

# Calculate percentage difference attributes.
voting_info_df['pctdiff_dem_vs_gop'] = (voting_info_df['votes_dem'] - voting_info_df['votes_gop']) / voting_info_df['totalvotes']
voting_info_df['pctdiff_gop_vs_dem'] = (voting_info_df['votes_gop'] - voting_info_df['votes_dem']) / voting_info_df['totalvotes']
voting_info_df['pctdiff_dem_vs_other'] = (voting_info_df['votes_dem'] - voting_info_df['votes_other']) / voting_info_df['totalvotes']
voting_info_df['pctdiff_gop_vs_other'] = (voting_info_df['votes_gop'] - voting_info_df['votes_other']) / voting_info_df['totalvotes']
voting_info_df['pctdiff_other_vs_dem'] = (voting_info_df['votes_other'] - voting_info_df['votes_dem']) / voting_info_df['totalvotes']
voting_info_df['pctdiff_other_vs_gop'] = (voting_info_df['votes_other'] - voting_info_df['votes_gop']) / voting_info_df['totalvotes']

# Calculate voter turnout attributes.
voting_info_df['voter_turnout'] = voting_info_df['totalvotes'] / voting_info_df['cvap_est']
voting_info_df['voter_turnout_majparty'] = (voting_info_df['votes_dem']+voting_info_df['votes_gop']) / voting_info_df['cvap_est']
voting_info_df['voter_turnout_dem'] = voting_info_df['votes_dem'] / voting_info_df['cvap_est']
voting_info_df['voter_turnout_gop'] = voting_info_df['votes_gop'] / voting_info_df['cvap_est']
voting_info_df['voter_turnout_other'] = voting_info_df['votes_other'] / voting_info_df['cvap_est']

# Determine the winning political party.
def return_winning_party(total_votes_dem, total_votes_gop, total_votes_other):
    if total_votes_dem > total_votes_gop and total_votes_dem > total_votes_other:
        return "Democratic Party"
    elif total_votes_gop > total_votes_dem and total_votes_gop > total_votes_other:
        return "Republican Party"
    elif total_votes_other > total_votes_dem and total_votes_other > total_votes_gop:
        return "Other Party"

voting_info_df["Winning Party"] = voting_info_df.apply(
    lambda x: return_winning_party(
        x.votes_dem,
        x.votes_gop,
        x.votes_other
    ), axis=1)

# Check the output.
voting_info_df.head()

<ipython-input-111-f17bd1b4e1ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voting_info_df['votes_other'] = voting_info_df['totalvotes'] - (voting_info_df['votes_dem'] + voting_info_df['votes_gop'])
<ipython-input-111-f17bd1b4e1ff>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voting_info_df['voter_share_major_party'] = (voting_info_df['votes_dem'] + voting_info_df['votes_gop']) / voting_info_df['totalvotes']
<ipython-input-111-f17bd1b4e1ff>:6: SettingWithCopyWarning: 
A value is trying to be set o

,year,FIPS,county_name,state,totalvotes,votes_dem,GREEN,LIBERTARIAN,OTHER,votes_gop,...,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,voter_turnout,voter_turnout_majparty,voter_turnout_dem,voter_turnout_gop,voter_turnout_other,Winning Party
6309,2008,1001.0,AUTAUGA,ALABAMA,23641,6093.0,NaN,NaN,145.0,17403.0,...,0.251597,0.730003,-0.251597,-0.730003,NaN,NaN,NaN,NaN,NaN,Republican Party
6310,2008,1003.0,BALDWIN,ALABAMA,81413,19386.0,NaN,NaN,756.0,61271.0,...,0.228833,0.743309,-0.228833,-0.743309,NaN,NaN,NaN,NaN,NaN,Republican Party
6311,2008,1005.0,BARBOUR,ALABAMA,11630,5697.0,NaN,NaN,67.0,5866.0,...,0.484093,0.498624,-0.484093,-0.498624,NaN,NaN,NaN,NaN,NaN,Republican Party
6312,2008,1007.0,BIBB,ALABAMA,8644,2299.0,NaN,NaN,83.0,6262.0,...,0.256363,0.714831,-0.256363,-0.714831,NaN,NaN,NaN,NaN,NaN,Republican Party
6313,2008,1009.0,BLOUNT,ALABAMA,24267,3522.0,NaN,NaN,356.0,20389.0,...,0.130465,0.825524,-0.130465,-0.825524,NaN,NaN,NaN,NaN,NaN,Republican Party


## Validate the data

Before continuing with other data preparation, you should confirm that the output data has been successfully created.

First, you will validate the values for voter turnout. Because these values represent a fraction (total votes divided by voting age population), you will confirm that the values range between 0 and 1.

In [112]:
voting_info_df['voter_turnout'].describe()

count    4.000000
mean     0.627225
std      0.030208
min      0.608909
25%      0.609674
50%      0.613930
75%      0.631480
max      0.672131
Name: voter_turnout, dtype: float64

The max value in the column is above 1, indicating a voter turnout above 100%. You will further investigate these anomalous records.

In [113]:
# Perform query for voter turnout above 100%.
turnout_over_1_df = voting_info_df.query('voter_turnout > 1')[['FIPS','county_name','state','year','voter_turnout','totalvotes','cvap_est']].sort_values(['FIPS', 'year'])
turnout_over_1_df

,FIPS,county_name,state,year,voter_turnout,totalvotes,cvap_est


All of these counties have small populations, which makes it harder to estimate their population of citizens of voting age with a high degree of accuracy. There is also a temporal mismatch between the two datasets used to calculate voter turnout. The estimates for citizens of voting age are compiled from American Community Survey results averaged over a five-year period. The number of votes was calculated at a specific point in time, which may not correspond to the five-year average.

There are three main ways you could address the impossibly high voter turnout values for these counties:

* Drop the records from the analysis.
* Keep the records as they are.
* Adjust the records to a plausible value.

Dropping the records means certain locations will have no data, making it harder to perform a spatial analysis. Keeping the current wrong values will reduce the quality of analysis results. Capping the voter turnout at 1 is a better measure of the actual voter turnout than the current values, especially since the small sizes of these counties indicate that the most likely source of the issue is an underestimate of the population of citizens of voting age.

You will adjust the voter_turnout value for each of these records to 1.

In [114]:
# Adjust values.
voting_info_df.loc[voting_info_df['voter_turnout'] > 1, 'voter_turnout'] = 1

# Describe data distribution.
voting_info_df['voter_turnout'].describe()

count    4.000000
mean     0.627225
std      0.030208
min      0.608909
25%      0.609674
50%      0.613930
75%      0.631480
max      0.672131
Name: voter_turnout, dtype: float64

## Create the modeling dataset using the 2020 election as response

In the current voting_info_df table, each record represents a county and a year. To analyze voter turnout in the 2020 election, you need a dataset where each county is represented only once. One option would be to drop data from previous elections from the table. But previous election turnout values might be useful in predicting turnout for 2020.  

You will pivot the table so that each county has one record and so that data for each election is in separate columns.

In [115]:
voting_info_pivot_df = voting_info_df.pivot(
    index=['FIPS'],
    columns=['year'],
    values=['totalvotes', 'cvap_est', 'voter_turnout', 'voter_turnout_dem', 'voter_turnout_gop', 'pctdiff_dem_vs_gop', 'rawdiff_dem_vs_gop', 'Winning Party'])

voting_info_pivot_df

totalvotes                          cvap_est                      \
year          2008    2012    2016    2020      2008      2012      2016   
FIPS                                                                       
1001.0       23641   23932   24973   27770       NaN       NaN       NaN   
1003.0       81413   85338   95215  109679       NaN       NaN       NaN   
1005.0       11630   11509   10469   10518       NaN       NaN       NaN   
1007.0        8644    8420    8819    9595       NaN       NaN       NaN   
1009.0       24267   24006   25588   27588       NaN       NaN       NaN   
...            ...     ...     ...     ...       ...       ...       ...   
56039.0      12359   11464   12627   14787       NaN       NaN       NaN   
56041.0       8418    8539    8470    9459       NaN       NaN       NaN   
56043.0       4100    3944    3814    4032       NaN       NaN       NaN   
56045.0       3414    3359    3526    3560       NaN       NaN       NaN   
11001       265853  293764  311268  344356  435875.0  475400.0  511190.0   

                  voter_turnout           ... pctdiff_dem_vs_gop            \
year         2020          2008     2012  ...               2016      2020   
FIPS                                      ...                                
1001.0        NaN           NaN      NaN  ...          -0.489969 -0.444184   
1003.0        NaN           NaN      NaN  ...          -0.571601 -0.537623   
1005.0        NaN           NaN      NaN  ...          -0.055688 -0.076631   
1007.0        NaN           NaN      NaN  ...          -0.551536  -0.57728   
1009.0        NaN           NaN      NaN  ...           -0.80909 -0.800022   
...           ...           ...      ...  ...                ...       ...   
56039.0       NaN           NaN      NaN  ...            0.26871  0.372422   
56041.0       NaN           NaN      NaN  ...          -0.584652 -0.624273   
56043.0       NaN           NaN      NaN  ...          -0.623755 -0.643353   
56045.0       NaN           NaN      NaN  ...          -0.775383 -0.771629   
11001    512335.0      0.609929  0.61793  ...           0.867763  0.867524   

        rawdiff_dem_vs_gop                                   Winning Party  \
year                  2008      2012      2016      2020              2008   
FIPS                                                                         
1001.0            -11310.0  -11016.0  -12236.0  -12335.0  Republican Party   
1003.0            -41885.0  -47592.0  -54425.0  -58966.0  Republican Party   
1005.0              -169.0     362.0    -583.0    -806.0  Republican Party   
1007.0             -3963.0   -3930.0   -4864.0   -5539.0  Republican Party   
1009.0            -16867.0  -17787.0  -20703.0  -22071.0  Republican Party   
...                    ...       ...       ...       ...               ...   
56039.0             2907.0    1355.0    3393.0    5507.0  Democratic Party   
56041.0            -3446.0   -4987.0   -4952.0   -5905.0  Republican Party   
56043.0            -1914.0   -2220.0   -2379.0   -2594.0  Republican Party   
56045.0            -1960.0   -2399.0   -2734.0   -2747.0  Republican Party   
11001             228433.0  245689.0  270107.0  298737.0  Democratic Party   

                                                               
year                 2012              2016              2020  
FIPS                                                           
1001.0   Republican Party  Republican Party  Republican Party  
1003.0   Republican Party  Republican Party  Republican Party  
1005.0   Democratic Party  Republican Party  Republican Party  
1007.0   Republican Party  Republican Party  Republican Party  
1009.0   Republican Party  Republican Party  Republican Party  
...                   ...               ...               ...  
56039.0  Democratic Party  Democratic Party  Democratic Party  
56041.0  Republican Party  Republican Party  Republican Party  
56043.0  Republican Party  Republican Party  Republic

If any counties did not have records for a particular election, the pivoted data frame will show null values for those years. You will check whether any records have null values in any field.

In [116]:
voting_info_pivot_df[voting_info_pivot_df.isna().any(axis=1)]

totalvotes                       cvap_est                 \
year          2008   2012   2016    2020     2008 2012 2016 2020   
FIPS                                                               
1001.0       23641  23932  24973   27770      NaN  NaN  NaN  NaN   
1003.0       81413  85338  95215  109679      NaN  NaN  NaN  NaN   
1005.0       11630  11509  10469   10518      NaN  NaN  NaN  NaN   
1007.0        8644   8420   8819    9595      NaN  NaN  NaN  NaN   
1009.0       24267  24006  25588   27588      NaN  NaN  NaN  NaN   
...            ...    ...    ...     ...      ...  ...  ...  ...   
56037.0      16833  16895  17130   16698      NaN  NaN  NaN  NaN   
56039.0      12359  11464  12627   14787      NaN  NaN  NaN  NaN   
56041.0       8418   8539   8470    9459      NaN  NaN  NaN  NaN   
56043.0       4100   3944   3814    4032      NaN  NaN  NaN  NaN   
56045.0       3414   3359   3526    3560      NaN  NaN  NaN  NaN   

        voter_turnout       ... pctdiff_dem_vs_gop            \
year             2008 2012  ...               2016      2020   
FIPS                        ...                                
1001.0            NaN  NaN  ...          -0.489969 -0.444184   
1003.0            NaN  NaN  ...          -0.571601 -0.537623   
1005.0            NaN  NaN  ...          -0.055688 -0.076631   
1007.0            NaN  NaN  ...          -0.551536  -0.57728   
1009.0            NaN  NaN  ...           -0.80909 -0.800022   
...               ...  ...  ...                ...       ...   
56037.0           NaN  NaN  ...          -0.520899 -0.503414   
56039.0           NaN  NaN  ...            0.26871  0.372422   
56041.0           NaN  NaN  ...          -0.584652 -0.624273   
56043.0           NaN  NaN  ...          -0.623755 -0.643353   
56045.0           NaN  NaN  ...          -0.775383 -0.771629   

        rawdiff_dem_vs_gop                                Winning Party  \
year                  2008     2012     2016     2020              2008   
FIPS                                                                      
1001.0            -11310.0 -11016.0 -12236.0 -12335.0  Republican Party   
1003.0            -41885.0 -47592.0 -54425.0 -58966.0  Republican Party   
1005.0              -169.0    362.0   -583.0   -806.0  Republican Party   
1007.0             -3963.0  -3930.0  -4864.0  -5539.0  Republican Party   
1009.0            -16867.0 -17787.0 -20703.0 -22071.0  Republican Party   
...                    ...      ...      ...      ...               ...   
56037.0            -4598.0  -6654.0  -8923.0  -8406.0  Republican Party   
56039.0             2907.0   1355.0   3393.0   5507.0  Democratic Party   
56041.0            -3446.0  -4987.0  -4952.0  -5905.0  Republican Party   
56043.0            -1914.0  -2220.0  -2379.0  -2594.0  Republican Party   
56045.0            -1960.0  -2399.0  -2734.0  -2747.0  Republican Party   

                                                               
year                 2012              2016              2020  
FIPS                                                           
1001.0   Republican Party  Republican Party  Republican Party  
1003.0   Republican Party  Republican Party  Republican Party  
1005.0   Democratic Party  Republican Party  Republican Party  
1007.0   Republican Party  Republican Party  Republican Party  
1009.0   Republican Party  Republican Party  Republican Party  
...                   ...               ...               ...  
56037.0  Republican Party  Republican Party  Republican Party  
56039.0  Democratic Party  Democratic Party  Democratic Party  
56041.0  Republican Party  Republican Party  Republican Party  
56043.0  Republican Party  Republican Party  Republican Party  
56045.0  Republican Party  Republican Party  Republican Party  

[3114 rows x 32 columns]

Additional research shows that Shannon County, SD (FIPS 46113), was renamed Oglala Lakota County in 2014 and assigned a new FIPS code (46102), which explains the missing data.

To facilitate analysis over time, you will replace the old FIPS code with the new one in the voting_info_df table and then repivot that dataframe.

In [117]:
# Make a copy of the data frame.
voting_info_fix_df = voting_info_df.copy()

# Fix the FIPS code for Oglala Lakota County.
voting_info_fix_df.loc[voting_info_fix_df['FIPS'] == '46113', 'FIPS'] = '46102'

# Pivot the table so that there is a single entry for each county.
voting_info_pivot_df = voting_info_fix_df.pivot(
    index=['FIPS'],
    columns=['year'],
    values=['totalvotes', 'cvap_est', 'voter_turnout', 'voter_turnout_dem', 'voter_turnout_gop', 'pctdiff_dem_vs_gop', 'rawdiff_dem_vs_gop', 'Winning Party'])

# Check the output to ensure there are no null values in any field.
voting_info_pivot_df[voting_info_pivot_df.isna().any(axis=1)]

totalvotes                       cvap_est                 \
year          2008   2012   2016    2020     2008 2012 2016 2020   
FIPS                                                               
1001.0       23641  23932  24973   27770      NaN  NaN  NaN  NaN   
1003.0       81413  85338  95215  109679      NaN  NaN  NaN  NaN   
1005.0       11630  11509  10469   10518      NaN  NaN  NaN  NaN   
1007.0        8644   8420   8819    9595      NaN  NaN  NaN  NaN   
1009.0       24267  24006  25588   27588      NaN  NaN  NaN  NaN   
...            ...    ...    ...     ...      ...  ...  ...  ...   
56037.0      16833  16895  17130   16698      NaN  NaN  NaN  NaN   
56039.0      12359  11464  12627   14787      NaN  NaN  NaN  NaN   
56041.0       8418   8539   8470    9459      NaN  NaN  NaN  NaN   
56043.0       4100   3944   3814    4032      NaN  NaN  NaN  NaN   
56045.0       3414   3359   3526    3560      NaN  NaN  NaN  NaN   

        voter_turnout       ... pctdiff_dem_vs_gop            \
year             2008 2012  ...               2016      2020   
FIPS                        ...                                
1001.0            NaN  NaN  ...          -0.489969 -0.444184   
1003.0            NaN  NaN  ...          -0.571601 -0.537623   
1005.0            NaN  NaN  ...          -0.055688 -0.076631   
1007.0            NaN  NaN  ...          -0.551536  -0.57728   
1009.0            NaN  NaN  ...           -0.80909 -0.800022   
...               ...  ...  ...                ...       ...   
56037.0           NaN  NaN  ...          -0.520899 -0.503414   
56039.0           NaN  NaN  ...            0.26871  0.372422   
56041.0           NaN  NaN  ...          -0.584652 -0.624273   
56043.0           NaN  NaN  ...          -0.623755 -0.643353   
56045.0           NaN  NaN  ...          -0.775383 -0.771629   

        rawdiff_dem_vs_gop                                Winning Party  \
year                  2008     2012     2016     2020              2008   
FIPS                                                                      
1001.0            -11310.0 -11016.0 -12236.0 -12335.0  Republican Party   
1003.0            -41885.0 -47592.0 -54425.0 -58966.0  Republican Party   
1005.0              -169.0    362.0   -583.0   -806.0  Republican Party   
1007.0             -3963.0  -3930.0  -4864.0  -5539.0  Republican Party   
1009.0            -16867.0 -17787.0 -20703.0 -22071.0  Republican Party   
...                    ...      ...      ...      ...               ...   
56037.0            -4598.0  -6654.0  -8923.0  -8406.0  Republican Party   
56039.0             2907.0   1355.0   3393.0   5507.0  Democratic Party   
56041.0            -3446.0  -4987.0  -4952.0  -5905.0  Republican Party   
56043.0            -1914.0  -2220.0  -2379.0  -2594.0  Republican Party   
56045.0            -1960.0  -2399.0  -2734.0  -2747.0  Republican Party   

                                                               
year                 2012              2016              2020  
FIPS                                                           
1001.0   Republican Party  Republican Party  Republican Party  
1003.0   Republican Party  Republican Party  Republican Party  
1005.0   Democratic Party  Republican Party  Republican Party  
1007.0   Republican Party  Republican Party  Republican Party  
1009.0   Republican Party  Republican Party  Republican Party  
...                   ...               ...               ...  
56037.0  Republican Party  Republican Party  Republican Party  
56039.0  Democratic Party  Democratic Party  Democratic Party  
56041.0  Republican Party  Republican Party  Republican Party  
56043.0  Republican Party  Republican Party  Republican Party  
56045.0  Republican Party  Republican Party  Republican Party  

[3114 rows x 32 columns]

The pivoted data frame has a multi-index for column names. You will flatten that multi-index and rename each column with the appropriate year.

In [118]:
voting_info_pivot_df.columns = [f"{a}_{b}" for a, b in voting_info_pivot_df.columns]
voting_info_pivot_df = voting_info_pivot_df.reset_index()
voting_info_pivot_df

,FIPS,totalvotes_2008,totalvotes_2012,totalvotes_2016,totalvotes_2020,cvap_est_2008,cvap_est_2012,cvap_est_2016,cvap_est_2020,voter_turnout_2008,...,pctdiff_dem_vs_gop_2016,pctdiff_dem_vs_gop_2020,rawdiff_dem_vs_gop_2008,rawdiff_dem_vs_gop_2012,rawdiff_dem_vs_gop_2016,rawdiff_dem_vs_gop_2020,Winning Party_2008,Winning Party_2012,Winning Party_2016,Winning Party_2020
0,1001.0,23641,23932,24973,27770,NaN,NaN,NaN,NaN,NaN,...,-0.489969,-0.444184,-11310.0,-11016.0,-12236.0,-12335.0,Republican Party,Republican Party,Republican Party,Republican Party
1,1003.0,81413,85338,95215,109679,NaN,NaN,NaN,NaN,NaN,...,-0.571601,-0.537623,-41885.0,-47592.0,-54425.0,-58966.0,Republican Party,Republican Party,Republican Party,Republican Party
2,1005.0,11630,11509,10469,10518,NaN,NaN,NaN,NaN,NaN,...,-0.055688,-0.076631,-169.0,362.0,-583.0,-806.0,Republican Party,Democratic Party,Republican Party,Republican Party
3,1007.0,8644,8420,8819,9595,NaN,NaN,NaN,NaN,NaN,...,-0.551536,-0.57728,-3963.0,-3930.0,-4864.0,-5539.0,Republican Party,Republican Party,Republican Party,Republican Party
4,1009.0,24267,24006,25588,27588,NaN,NaN,NaN,NaN,NaN,...,-0.80909,-0.800022,-16867.0,-17787.0,-20703.0,-22071.0,Republican Party,Republican Party,Republican Party,Republican Party
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3110,56039.0,12359,11464,12627,14787,NaN,NaN,NaN,NaN,NaN,...,0.26871,0.372422,2907.0,1355.0,3393.0,5507.0,Democratic Party,Democratic Party,Democratic Party,Democratic Party
3111,56041.0,8418,8539,8470,9459,NaN,NaN,NaN,NaN,NaN,...,-0.584652,-0.624273,-3446.0,-4987.0,-4952.0,-5905.0,Republican Party,Republican Party,Republican Party,Republican Party
3112,56043.0,4100,3944,3814,4032,NaN,NaN,NaN,NaN,NaN,...,-0.623755,-0.643353,-1914.0,-2220.0,-2379.0,-2594.0,Republican Party,Republican Party,Republican Party,Republican Party
3113,56045.0,3414,3359,3526,3560,NaN,NaN,NaN,NaN,NaN,...,-0.775383,-0.771629,-1960.0,-2399.0,-2734.0,-2747.0,Republican Party,Republican Party,Republican Party,Republican Party


In [123]:

voting_info_pivot_df['FIPS'] = voting_info_pivot_df['FIPS'].astype(int)
voting_info_pivot_df

,FIPS,totalvotes_2008,totalvotes_2012,totalvotes_2016,totalvotes_2020,cvap_est_2008,cvap_est_2012,cvap_est_2016,cvap_est_2020,voter_turnout_2008,...,pctdiff_dem_vs_gop_2016,pctdiff_dem_vs_gop_2020,rawdiff_dem_vs_gop_2008,rawdiff_dem_vs_gop_2012,rawdiff_dem_vs_gop_2016,rawdiff_dem_vs_gop_2020,Winning Party_2008,Winning Party_2012,Winning Party_2016,Winning Party_2020
0,1001,23641,23932,24973,27770,NaN,NaN,NaN,NaN,NaN,...,-0.489969,-0.444184,-11310.0,-11016.0,-12236.0,-12335.0,Republican Party,Republican Party,Republican Party,Republican Party
1,1003,81413,85338,95215,109679,NaN,NaN,NaN,NaN,NaN,...,-0.571601,-0.537623,-41885.0,-47592.0,-54425.0,-58966.0,Republican Party,Republican Party,Republican Party,Republican Party
2,1005,11630,11509,10469,10518,NaN,NaN,NaN,NaN,NaN,...,-0.055688,-0.076631,-169.0,362.0,-583.0,-806.0,Republican Party,Democratic Party,Republican Party,Republican Party
3,1007,8644,8420,8819,9595,NaN,NaN,NaN,NaN,NaN,...,-0.551536,-0.57728,-3963.0,-3930.0,-4864.0,-5539.0,Republican Party,Republican Party,Republican Party,Republican Party
4,1009,24267,24006,25588,27588,NaN,NaN,NaN,NaN,NaN,...,-0.80909,-0.800022,-16867.0,-17787.0,-20703.0,-22071.0,Republican Party,Republican Party,Republican Party,Republican Party
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3110,56039,12359,11464,12627,14787,NaN,NaN,NaN,NaN,NaN,...,0.26871,0.372422,2907.0,1355.0,3393.0,5507.0,Democratic Party,Democratic Party,Democratic Party,Democratic Party
3111,56041,8418,8539,8470,9459,NaN,NaN,NaN,NaN,NaN,...,-0.584652,-0.624273,-3446.0,-4987.0,-4952.0,-5905.0,Republican Party,Republican Party,Republican Party,Republican Party
3112,56043,4100,3944,3814,4032,NaN,NaN,NaN,NaN,NaN,...,-0.623755,-0.643353,-1914.0,-2220.0,-2379.0,-2594.0,Republican Party,Republican Party,Republican Party,Republican Party
3113,56045,3414,3359,3526,3560,NaN,NaN,NaN,NaN,NaN,...,-0.775383,-0.771629,-1960.0,-2399.0,-2734.0,-2747.0,Republican Party,Republican Party,Republican Party,Republican Party


***

## Geoenable the data

To use this data in a spatial analysis, the data needs to include location information to determine where each county is located on a map. This step geocodes the data by joining it to existing county geometries.

There are various resources that you can use to find geoenabled data. [ArcGIS Living Atlas of the World](https://livingatlas.arcgis.com) is an authoritative source provided by Esri. Each record in your election data represents information for a county, so you will use an ArcGIS Living Atlas dataset that represents county geometry.

ArcGIS API for Python, which you imported at the beginning of this notebook, extends the pandas data frame to allow spatially enabled data frames that can hold geometry. Converting the feature layer from ArcGIS Living Atlas to a spatially enabled data frame will allow you to join that geometry to the voting data.

**Note: Executing the following cell may take a few minutes.**

In [ ]:
# The relevant layer is available to the public, so you can connect to ArcGIS Online anonymously.
#gis = arcgis.gis.GIS()

# Get the USA Census counties layer from ArcGIS Living Atlas.
#item = gis.content.get('14c5450526a8430298b2fa74da12c2f4')

# Convert the layer to a spatially enabled data frame.
#counties_sdf = pd.DataFrame.spatial.from_layer(item.layers[0])
#counties_sdf.head()

In [125]:
counties_sdf = gpd.read_file('/content/USA_Census_Counties.zip')
counties_sdf['FIPS'] = counties_sdf['FIPS'].astype(int)
counties_sdf.head()

,NAME,STATE_NAME,STATE_ABBR,STATE_FIPS,COUNTY_FIP,FIPS,POPULATION,POP_SQMI,SQMI,Shape__Are,Shape__Len,geometry
0,Autauga County,Alabama,AL,01,001,1001,58805,97.3,604.37,0.150256,2.066033,"POLYGON ((-86.41312 32.70739, -86.41300 32.678..."
1,Baldwin County,Alabama,AL,01,003,1003,231767,141.9,1633.14,0.398404,9.305629,"MULTIPOLYGON (((-87.56491 30.28162, -87.56470 ..."
2,Barbour County,Alabama,AL,01,005,1005,25223,27.9,904.52,0.223270,2.695260,"POLYGON ((-85.25784 32.14794, -85.25851 32.146..."
3,Bibb County,Alabama,AL,01,007,1007,22293,35.6,626.17,0.156473,1.887519,"POLYGON ((-87.06574 33.24691, -87.06477 33.246..."
4,Blount County,Alabama,AL,01,009,1009,59134,90.9,650.63,0.164405,2.423466,"POLYGON ((-86.45302 34.25932, -86.45288 34.259..."


The SHAPE field in the spatially enabled data frame represents the county geometry and is used to locate each record, or feature, on the map. You will join that geometry with the voting information you created earlier.

In [126]:
geo_sdf = counties_sdf.merge(voting_info_pivot_df, on='FIPS')
geo_sdf.head()

,NAME,STATE_NAME,STATE_ABBR,STATE_FIPS,COUNTY_FIP,FIPS,POPULATION,POP_SQMI,SQMI,Shape__Are,...,pctdiff_dem_vs_gop_2016,pctdiff_dem_vs_gop_2020,rawdiff_dem_vs_gop_2008,rawdiff_dem_vs_gop_2012,rawdiff_dem_vs_gop_2016,rawdiff_dem_vs_gop_2020,Winning Party_2008,Winning Party_2012,Winning Party_2016,Winning Party_2020
0,Autauga County,Alabama,AL,01,001,1001,58805,97.3,604.37,0.150256,...,-0.489969,-0.444184,-11310.0,-11016.0,-12236.0,-12335.0,Republican Party,Republican Party,Republican Party,Republican Party
1,Baldwin County,Alabama,AL,01,003,1003,231767,141.9,1633.14,0.398404,...,-0.571601,-0.537623,-41885.0,-47592.0,-54425.0,-58966.0,Republican Party,Republican Party,Republican Party,Republican Party
2,Barbour County,Alabama,AL,01,005,1005,25223,27.9,904.52,0.223270,...,-0.055688,-0.076631,-169.0,362.0,-583.0,-806.0,Republican Party,Democratic Party,Republican Party,Republican Party
3,Bibb County,Alabama,AL,01,007,1007,22293,35.6,626.17,0.156473,...,-0.551536,-0.57728,-3963.0,-3930.0,-4864.0,-5539.0,Republican Party,Republican Party,Republican Party,Republican Party
4,Blount County,Alabama,AL,01,009,1009,59134,90.9,650.63,0.164405,...,-0.80909,-0.800022,-16867.0,-17787.0,-20703.0,-22071.0,Republican Party,Republican Party,Republican Party,Republican Party


In [128]:
# Join the voting information dataframe with the counties geometry.
geo_sdf = pd.merge(counties_sdf, voting_info_pivot_df, left_on='FIPS', right_on='FIPS', how='right')
geo_sdf.head()

,NAME,STATE_NAME,STATE_ABBR,STATE_FIPS,COUNTY_FIP,FIPS,POPULATION,POP_SQMI,SQMI,Shape__Are,...,pctdiff_dem_vs_gop_2016,pctdiff_dem_vs_gop_2020,rawdiff_dem_vs_gop_2008,rawdiff_dem_vs_gop_2012,rawdiff_dem_vs_gop_2016,rawdiff_dem_vs_gop_2020,Winning Party_2008,Winning Party_2012,Winning Party_2016,Winning Party_2020
0,Autauga County,Alabama,AL,01,001,1001,58805.0,97.3,604.37,0.150256,...,-0.489969,-0.444184,-11310.0,-11016.0,-12236.0,-12335.0,Republican Party,Republican Party,Republican Party,Republican Party
1,Baldwin County,Alabama,AL,01,003,1003,231767.0,141.9,1633.14,0.398404,...,-0.571601,-0.537623,-41885.0,-47592.0,-54425.0,-58966.0,Republican Party,Republican Party,Republican Party,Republican Party
2,Barbour County,Alabama,AL,01,005,1005,25223.0,27.9,904.52,0.223270,...,-0.055688,-0.076631,-169.0,362.0,-583.0,-806.0,Republican Party,Democratic Party,Republican Party,Republican Party
3,Bibb County,Alabama,AL,01,007,1007,22293.0,35.6,626.17,0.156473,...,-0.551536,-0.57728,-3963.0,-3930.0,-4864.0,-5539.0,Republican Party,Republican Party,Republican Party,Republican Party
4,Blount County,Alabama,AL,01,009,1009,59134.0,90.9,650.63,0.164405,...,-0.80909,-0.800022,-16867.0,-17787.0,-20703.0,-22071.0,Republican Party,Republican Party,Republican Party,Republican Party


The merge used a right join to ensure that every record in the election data is included in the output. However, with a right join, it is possible that election records for some counties were not joined to a geometry. To confirm that the join was successful, you will check to see whether any record has null geometry.

In [129]:
geo_sdf.query("geometry.isnull()")

,NAME,STATE_NAME,STATE_ABBR,STATE_FIPS,COUNTY_FIP,FIPS,POPULATION,POP_SQMI,SQMI,Shape__Are,...,pctdiff_dem_vs_gop_2016,pctdiff_dem_vs_gop_2020,rawdiff_dem_vs_gop_2008,rawdiff_dem_vs_gop_2012,rawdiff_dem_vs_gop_2016,rawdiff_dem_vs_gop_2020,Winning Party_2008,Winning Party_2012,Winning Party_2016,Winning Party_2020
1797,NaN,NaN,NaN,NaN,NaN,36000,NaN,NaN,NaN,NaN,...,0.568277,0.594731,88248.0,76161.0,73081.0,81267.0,Democratic Party,Democratic Party,Democratic Party,Democratic Party
2388,NaN,NaN,NaN,NaN,NaN,46113,NaN,NaN,NaN,NaN,...,NaN,NaN,2640.0,2749.0,NaN,NaN,Democratic Party,Democratic Party,NaN,NaN
2887,NaN,NaN,NaN,NaN,NaN,51515,NaN,NaN,NaN,NaN,...,NaN,NaN,-289.0,-302.0,0.0,NaN,Republican Party,Republican Party,None,NaN


Because no record has null geometry, you know that every voting record was joined to a county.

## Convert data frames to feature classes

##### Define the ArcGIS Pro project, map, workspace, and database

In [ ]:
# Create variables that represent the ArcGIS Pro project and map.
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps('Data Engineering')[0]

# Create a variable that represents the default file geodatabase.
arcpy.env.workspace = aprx.defaultGeodatabase
arcpy.env.addOutputsToMap = False

In [ ]:
arcpy.env.workspace

The pandas extension provided by the ArcGIS API for Python also enables exporting a spatially enabled data frame to a feature class.

**Note: Executing the following cell may take a few minutes**

In [ ]:
# Create a feature class for the 2020 presidential election.
out_fc_name = "county_elections_pres"
out_fc_path = os.path.join(arcpy.env.workspace, out_fc_name)
out_fc = geo_sdf.spatial.to_featureclass(out_fc_path)
out_fc

You used an ArcGIS notebook to complete the necessary data engineering tasks. Data engineering was needed to prepare this data for a predictive analysis that will model voter turnout using demographic variables, such as per capita income.


## Return to the exercise and go to the Modify Environment Settings step for the remaining instructions.


***